In [ ]:
from __future__ import absolute_import, division, print_function
%matplotlib inline

import os
import numpy as np
import PIL.Image as pil
import matplotlib.pyplot as plt

import torch
from torchvision import transforms

import networks
from utils import download_model_if_doesnt_exist
from time import sleep


from djitellopy import tello
import time
import cv2
import numpy as np
from socket import *
import sys
import pygame
from PIL import Image

import glob
import argparse
import matplotlib
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '2'
from keras.models import load_model
from matplotlib import pyplot as plt
from keras.layers import Layer, InputSpec
import keras.utils.conv_utils as conv_utils
import tensorflow as tf
import keras.backend as K
import pandas as pd

from djitellopy import tello
import cv2 
import uuid
import os
import time
import wget
import numpy as np
import tensorflow as tf
from google.protobuf import text_format
from Tensorflow import config_util
from Tensorflow import label_map_util
from Tensorflow import visualization_utils as viz_utils
from Tensorflow import New
from Tensorflow import model_builder
from matplotlib import pyplot as plt
%matplotlib inline

In [ ]:
from djitellopy import tello

drone = tello.Tello()
drone.connect()
print(drone.get_battery())

In [ ]:
def DepthNorm(x, maxDepth):
    return maxDepth / x

def predict(model, images, minDepth=10, maxDepth=1000, batch_size=2):
    # Support multiple RGBs, one RGB image, even grayscale Z
    if len(images.shape) < 3: images = np.stack((images, images, images), axis=2)
    if len(images.shape) < 4: images = images.reshape((1, images.shape[0], images.shape[1], images.shape[2]))
    # Compute predictions
    predictions = model.predict(images, batch_size=batch_size)
    # Put in expected range
    return np.clip(DepthNorm(predictions, maxDepth=maxDepth), minDepth, maxDepth) / maxDepth

def scale_up(scale, images):
    pass

def load_images(image_files):  # required
    loaded_images = []
    for file in image_files:
        x = np.clip(np.asarray(Image.open(file), dtype=float) / 255, 0, 1)
        loaded_images.append(x)
    return np.stack(loaded_images, axis=0)

def to_multichannel(i):
    if i.shape[2] == 3: return i
    i = i[:, :, 0]
    return np.stack((i, i, i), axis=2)

def display_images(outputs, inputs=None, gt=None, is_colormap=True, is_rescale=True):
    import matplotlib.pyplot as plt
    import skimage
    from skimage.transform import resize
    plasma = plt.get_cmap('plasma')
    shape = (outputs[0].shape[0], outputs[0].shape[1], 3)
    all_images = []
    for i in range(outputs.shape[0]):
        print(i)
        print(range(outputs.shape[0]))
        imgs = []
        if isinstance(gt, (list, tuple, np.ndarray)):
            x = to_multichannel(gt[i])
            x = resize(x, shape, preserve_range=True, mode='reflect', anti_aliasing=True)
            imgs.append(x)
        if is_colormap:
            rescaled = outputs[i][:, :, 0]
            if is_rescale:
                rescaled = rescaled - np.min(rescaled)
                rescaled = rescaled / np.max(rescaled)
            imgs.append(plasma(rescaled)[:, :, :3])
        else:
            imgs.append(to_multichannel(outputs[i]))
        img_set = np.hstack(imgs)
        all_images.append(img_set)
    all_images = np.stack(all_images)
    return skimage.util.montage(all_images, multichannel=True, fill=(0, 0, 0))

def evaluate(model, rgb, depth, crop, batch_size=6, verbose=False):
    pass

def compute_errors(gt, pred):
    pass

def load_test_data(test_data_zip_file='nyu_test.zip'):
    pass

def save_images(filename, outputs, inputs=None, gt=None, is_colormap=True, is_rescale=False):
    pass

class BilinearUpSampling2D(Layer):
    def __init__(self, size=(2, 2), data_format=None, **kwargs):
        super(BilinearUpSampling2D, self).__init__(**kwargs)
        self.data_format = K.image_data_format().lower()
        self.size = conv_utils.normalize_tuple(size, 2, 'size')
        self.input_spec = InputSpec(ndim=4)
    def compute_output_shape(self, input_shape):
        if self.data_format == 'channels_first':
            height = self.size[0] * input_shape[2] if input_shape[2] is not None else None
            width = self.size[1] * input_shape[3] if input_shape[3] is not None else None
            return (input_shape[0], input_shape[1], height, width)
        elif self.data_format == 'channels_last':
            height = self.size[0] * input_shape[1] if input_shape[1] is not None else None
            width = self.size[1] * input_shape[2] if input_shape[2] is not None else None
            return (input_shape[0], height, width, input_shape[3])
    def call(self, inputs):
        input_shape = K.shape(inputs)
        if self.data_format == 'channels_first':
            height = self.size[0] * input_shape[2] if input_shape[2] is not None else None
            width = self.size[1] * input_shape[3] if input_shape[3] is not None else None
        elif self.data_format == 'channels_last':
            height = self.size[0] * input_shape[1] if input_shape[1] is not None else None
            width = self.size[1] * input_shape[2] if input_shape[2] is not None else None
        return tf.image.resize(inputs, [height, width], method=tf.image.ResizeMethod.BILINEAR)
    def get_config(self):
        config = {'size': self.size, 'data_format': self.data_format}
        base_config = super(BilinearUpSampling2D, self).get_config()
        return dict(list(base_config.items()) + list(config.items()))

In [ ]:
parser = argparse.ArgumentParser()
parser.add_argument('--model', default='nyu.h5', type=str, help='Trained Keras model file.')
parser.add_argument('--input', default='centered.png', type=str, help='Input filename or folder.')
args, unknown = parser.parse_known_args()

custom_objects = {'BilinearUpSampling2D': BilinearUpSampling2D, 'depth_loss_function': None}

K.clear_session()

model = load_model(args.model, custom_objects=custom_objects, compile=False)

In [ ]:
def Dense():
    x=time.time()
    inputs = load_images(glob.glob(args.input))
    outputs = predict(model,inputs)
    xx=time.time()
    print("Time delay of depth calculation is ",xx-x)
    return outputs

In [ ]:
CUSTOM_MODEL_NAME = 'model' 
LABEL_MAP_NAME = 'label_map.pbtxt'

paths = {
    'ANNOTATION_PATH': os.path.join('Tensorflow'),
    'IMAGE_PATH': os.path.join('Tensorflow','images'),
    'CHECKPOINT_PATH': os.path.join('Tensorflow',CUSTOM_MODEL_NAME),
 }

files = {
    'PIPELINE_CONFIG':os.path.join('Tensorflow', CUSTOM_MODEL_NAME, 'pipeline.config'),
    'LABELMAP': os.path.join(paths['ANNOTATION_PATH'], LABEL_MAP_NAME)
}

In [ ]:
# Load pipeline config and build a detection model
configs = config_util.get_configs_from_pipeline_file(files['PIPELINE_CONFIG'])
detection_model = model_builder.build(model_config=configs['model'], is_training=False)

# Restore checkpoint
ckpt = tf.compat.v2.train.Checkpoint(model=detection_model)
ckpt.restore(os.path.join(paths['CHECKPOINT_PATH'], 'ckpt-3')).expect_partial()

@tf.function
def detect_fn(image):
    image, shapes = detection_model.preprocess(image)
    prediction_dict = detection_model.predict(image, shapes)
    detections = detection_model.postprocess(prediction_dict, shapes)
    return detections

category_index = label_map_util.create_category_index_from_labelmap(files['LABELMAP'])

In [ ]:
# Real Time Detections

drone.streamon()
time.sleep(1)
drone.takeoff()
# time.sleep(3)
drone.send_rc_control(0,0,0,0)
width = 640 # Do not change this because calculations are built on this value #int(cap.get(cv2.CAP_PROP_FRAME_WIDTH)) 
height = 480 # Do not change this because calculations are built on this value #int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
rl,fb,ud,y = 0, 0, 0, 0
speed = 18
angle = drone.get_yaw()
print (angle)
t1 = time.time()
k = 0
while True:
    print ("new loop started")
    i = 0
    frame = drone.get_frame_read().frame
    frame = cv2.resize(frame, (640, 480))
    image_np = np.array(frame)
    
    input_tensor = tf.convert_to_tensor(np.expand_dims(image_np, 0), dtype=tf.float32)
    detections = detect_fn(input_tensor)
    
    num_detections = int(detections.pop('num_detections'))
    detections = {key: value[0, :num_detections].numpy()
                  for key, value in detections.items()}
    detections['num_detections'] = num_detections

    # detection_classes should be ints.
    detections['detection_classes'] = detections['detection_classes'].astype(np.int64)

    label_id_offset = 1
    image_np_with_detections = image_np.copy()

    _, ymin, xmin, ymax, xmax = New.visualize_boxes_and_labels_on_image_array(
                image_np_with_detections,
                detections['detection_boxes'],
                detections['detection_classes']+label_id_offset,
                detections['detection_scores'],
                category_index,
                use_normalized_coordinates=True,
                max_boxes_to_draw=5,
                min_score_thresh=.53,
                agnostic_mode=False)
    
    img = cv2.resize(image_np_with_detections, (640, 480))
    cv2.imshow('object detection', img)

    if ymin != 0:
        print ("detection")
        i, j = 0, 0 
        ymin = int(ymin*480)
        ymax = int(ymax*480)
        xmin = int(xmin*640)
        xmax = int(xmax*640)
        
        xlength = xmax - xmin
        ylength = ymax - ymin
        
        Xcenter_point = (xmax + xmin)/2 
        Ycenter_point = (ymax + ymin)/2

        delta = np.array([Xcenter_point, Ycenter_point]) - np.array([width/2, height/2])
        BArea = xlength * ylength # object detected Box area
        IArea = width * height # image area from the drone camera assuming width always = 640 and height = 480
        RArea = BArea/IArea*100 # box to image area ratio
        print (RArea)
        
        if abs(RArea) < 2.5:
            fb = speed # forward/backward
            drone.send_rc_control(rl,fb,ud,y)
        
        elif abs(RArea) < 6 and abs(RArea) > 2.5:
            fb = 0
            drone.send_rc_control(rl,fb,ud,y)
            if abs(delta[0]) > 5:
                if delta[0] > 0:
                    rl = speed
                    drone.send_rc_control(rl,fb,ud,y)
                else:
                    rl = -speed
                    drone.send_rc_control(rl,fb,ud,y)
            else:
                rl = 0
                i = 1
                drone.send_rc_control(rl,fb,ud,y)
                    
            if abs(delta[1]) > 5:
                if delta[1] > 0:
                    ud = -speed
                    drone.send_rc_control(rl,fb,ud,y)
                else:
                    ud = speed
                    drone.send_rc_control(rl,fb,ud,y)
            else: 
                ud = 0
                j = 1
                drone.send_rc_control(rl,fb,ud,y)
                
            if i == 1 and j == 1:
                t2 = time.time()
                cv2.imwrite('centered.png', frame)
                drone.send_rc_control(0,0,0,0)
                outputs = Dense()
                x = outputs[0][119][159][0]
                outputs_1 = x + 0.1303 * x
                outputs_2 = x - 0.1303 * x
                
                print ("The object is in the range of ", outputs_1, " and ", outputs_2)
                print ("The total time from first detection to centering is", t2-t1)
                drone.land()
                sleep(5)
                
        elif abs(RArea) > 6:
            fb = - speed
            drone.send_rc_control(rl,fb,ud,y)
    
    else: 
        drone.send_rc_control(0,0,0,0)
        print ("no detection")
        sleep(0.1)

    if cv2.waitKey(10) & 0xFF == ord('q'):
        #cv2.destroyallwindows()
        break